In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, length
spark = SparkSession.builder \
    .master("spark://192.168.2.219:7077") \
    .appName("YourAppName") \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/08 12:09:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data_path = "hdfs://192.168.2.219:9001/user/root/reddit_data/corpus-webis-tldr-17.json"
df_old = spark.read.json(data_path)

In [ ]:
df = df_old.limit(1000)

In [ ]:
df.cache()
nrows = df.count() # force caching
    
# need to access hidden parameters from the `SparkSession` and `DataFrame`
size_bytes = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(df._jdf)

In [14]:
df = df.withColumn("content_length", length("content"))
df = df.withColumn("summary_length", length("summary"))
# Group by subreddit and calculate average lengths
average_lengths = df.groupBy("subreddit").agg(
    avg("content_length").alias("avg_content_length"),
    avg("summary_length").alias("avg_summary_length")
)
# Order by subreddit for readability (optional, can be removed for large datasets)
average_lengths = average_lengths.orderBy("subreddit")
# Show the results
average_lengths.show()
# Optionally, save the results to a file
output_path = "/home/ubuntu/Drive/reddit_data/output/average_lengths.csv"
average_lengths.write.csv(output_path, header=True)

+-------------------+------------------+------------------+
|          subreddit|avg_content_length|avg_summary_length|
+-------------------+------------------+------------------+
|              0x10c|1034.4545454545455| 243.1818181818182|
|         0x10cships|             869.0|              44.0|
|1000thworldproblems|              14.0|               6.0|
|         100pushups|             578.0|              67.5|
|            100sets|            2692.0|              44.0|
|       1200isplenty|1109.8333333333333|238.16666666666666|
|            13thage|            1622.0|             153.0|
|     14TeamStandard|             475.0|              27.0|
|               1911|             868.4|             121.4|
|          2007scape| 704.9596412556053| 93.37219730941705|
|      2012Elections|             970.2|              52.0|
|      2020elections|            3009.0|             448.0|
|               2048|            2276.0|              58.5|
|           20xxgame|             434.0|

In [ ]:
average_lenghts

24/03/08 12:06:22 WARN TransportChannelHandler: Exception in connection from /192.168.2.219:55494
java.io.IOException: Connection reset by peer
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoo